### instalar dependencias

In [10]:
! pip install pandas
! pip install requests
! pip install beautifulsoup4
! pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/162.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/162.8 kB ? eta -:--:--
     -------------- ------------------------ 61.4/162.8 kB 3.2 MB/s eta 0:00:01
     -------------------------- ----------- 112.6/162.8 kB 1.6 MB/s eta 0:00:01
     -------------------------------------- 162.8/162.8 kB 1.4 MB/s eta 0:00:00
  Using cached kiwisolver-1.4.5-cp312-cp312-win_amd64.whl.metadata (6.5 kB)
  Using cached pillow-10.3.0-cp312-cp312-win_amd64.whl.metadata (9.4 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
    --------------------------------------- 0.1/8.0 MB 4.3 MB/s eta 0:00:02
   - -------------------------------------- 0.4/8.0 MB 4.6 MB/s eta 0:00:02
   ---- ----------------------------------- 1.0/8.0 MB 7.9 MB/s eta 0:00:01
   -------- --------------------------

### Obtener data

In [11]:
import requests
import json

try:
    with open("jobs.json", 'r', encoding='utf-8') as file:
        jobs = json.load(file)

except FileNotFoundError:
    response = requests.get("https://www.getonbrd.com/api/v0/search/jobs", params={
        "query": "Backend",
        "per_page": 100,
        "page": 1,
    })

    jobs = response.json()["data"]

    with open("jobs.json", 'w', encoding='utf-8') as file:
        json.dump(jobs, file, ensure_ascii=False, indent=4)
        



### guardar data

In [12]:
import json

def guardar_en_json(array_de_objetos, nombre_archivo):
    
    with open(nombre_archivo, 'w', encoding='utf-8') as archivo_json:
        json.dump(array_de_objetos, archivo_json, ensure_ascii=False, indent=4)


guardar_en_json(jobs, 'jobs.json')

### Construir data frame

In [19]:
import pandas as pd
from bs4 import BeautifulSoup

def transform_jobs_to_dataframe(jobs):
    
    structured_data = []
    for job in jobs:
        job_data = {
            "id": job["id"],
            "title": job["attributes"]["title"],
            "description": BeautifulSoup(job["attributes"].get("description", ""), 'html.parser').get_text(),
            "projects": BeautifulSoup(job["attributes"].get("projects", ""), 'html.parser').get_text(),
            "functions": BeautifulSoup(job["attributes"].get("functions", ""), 'html.parser').get_text(),
            "benefits": BeautifulSoup(job["attributes"].get("benefits", ""), 'html.parser').get_text(),
            "desirable": BeautifulSoup(job["attributes"].get("desirable", ""), 'html.parser').get_text(),
            "seniority": job["attributes"].get("seniority", ""),
            "remote": job["attributes"]["remote"],
            "remote_modality": job["attributes"].get("remote_modality", ""),
            "remote_zone": job["attributes"].get("remote_zone", ""),
            "countries": job["attributes"]["countries"],
            "lang": job["attributes"]["lang"],
            "category_name": job["attributes"]["category_name"],
            "perks": job["attributes"]["perks"],
            "min_salary": job["attributes"].get("min_salary"),
            "max_salary": job["attributes"].get("max_salary"),
            "published_at": job["attributes"]["published_at"],
            "response_time_min": job["attributes"]["response_time_in_days"].get("min"),
            "response_time_max": job["attributes"]["response_time_in_days"].get("max"),
            "applications_count": job["attributes"]["applications_count"],
            "public_url": job["links"]["public_url"]
        }
        structured_data.append(job_data)
    
    return pd.DataFrame(structured_data)

df_jobs = transform_jobs_to_dataframe(jobs)
df_jobs

def get_seniority_type(seniority):
    return seniority['data']['type']

df_jobs['seniority_type'] = df_jobs['seniority'].apply(get_seniority_type)

seniority_index = df_jobs.columns.get_loc('seniority')

df_jobs.insert(seniority_index + 1, 'seniority_type', df_jobs.pop('seniority_type'))

df_jobs



,id,title,description,projects,functions,benefits,desirable,seniority,seniority_type,remote,...,lang,category_name,perks,min_salary,max_salary,published_at,response_time_min,response_time_max,applications_count,public_url
0,desarrollador-a-backend-javase-spring-boot-aws...,Desarrollador/a Back-end Javase Spring Boot AWS,"¿Qué requerimos?Técnico, Tecnólogo o Profesion...",Acerca del empleoSophos Solutions es una compa...,Tendrás la oportunidad de:Desarrollar en Java ...,Nosotros te ofrecemos 💜Contrato a término inde...,Es un Plus:Experiencia/Conocimientos en Mongo ...,"{'data': {'id': 3, 'type': 'seniority'}}",seniority,True,...,lang_not_specified,SysAdmin / DevOps / QA,"[wellness, digital_library, health_coverage, c...",NaN,NaN,1716475444,NaN,NaN,15,https://www.getonbrd.com/jobs/desarrollador-a-...
1,tech-lead-arkho-santiago-70da,Data Tech Lead,¿Cuáles son los requerimientos del cargo? Expe...,ARKHO es una consultora experta en tecnologías...,Estamos en busca de un profesional con experie...,Día administrativo semestral hasta los 12 mese...,"Back: Microservicios, API devolopmentManejo de...","{'data': {'id': 3, 'type': 'seniority'}}",seniority,True,...,lang_not_specified,Programming,"[remote_full, flexible_hours, informal_dressco...",NaN,NaN,1716436734,5.0,9.0,172,https://www.getonbrd.com/jobs/tech-lead-arkho-...
2,desarrollador-a-backend-springboot-empresas-sb...,Desarrollador/a Back-end (Spring Boot),Experiencia en desarrollo de soluciones inform...,"Somos un grupo de empresas de retail, especial...",¿Cuál será tu desafío en este cargo?Crear o ma...,,,"{'data': {'id': 4, 'type': 'seniority'}}",seniority,False,...,lang_not_specified,Programming,"[wellness, accessible, meals_provided, commuti...",NaN,NaN,1716413884,NaN,NaN,34,https://www.getonbrd.com/jobs/desarrollador-a-...
3,machine-learning-engineer-neuralworks-santiago...,Machine Learning Engineer,Ingeniería Civil en Computación o similarAl me...,NeuralWorks es una compañía de alto crecimient...,El equipo de analytics trabaja en diferentes p...,MacBook Air M1 o similar (con opción de compra...,"Experiencia en Infraestructura as code, observ...","{'data': {'id': 3, 'type': 'seniority'}}",seniority,False,...,es,Machine Learning & AI,"[library, accessible, flexible_hours, internal...",NaN,NaN,1716241209,NaN,NaN,55,https://www.getonbrd.com/jobs/machine-learning...
4,arquitecto-java-bilingue-kibernum-bogota,Arquitecto Java Bilingüe,RequisitosProfesional en ingeniería de sistema...,"Kibernum, es una de las empresas líderes en te...","Responsable del diseño, arquitectura y documen...",Tipo de Trabajo: Remoto Tipo de Contrato: Inde...,,"{'data': {'id': 4, 'type': 'seniority'}}",seniority,True,...,en,Programming,[computer_provided],NaN,NaN,1715899810,NaN,NaN,9,https://www.getonbrd.com/jobs/arquitecto-java-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,analista-qa-automatizador-iconstruye-remote,Analista QA Automatizador,Conocimientos de Ingeniería del Software.Conoc...,¡Súmate a nuestro equipo! Estamos buscando a n...,"Analizar Requerimientos, Historias de Usuario,...",5 días extras de descanso.Tarjeta amipass para...,,"{'data': {'id': 3, 'type': 'seniority'}}",seniority,False,...,lang_not_specified,SysAdmin / DevOps / QA,"[flexible_hours, computer_provided, informal_d...",NaN,NaN,1715187574,NaN,NaN,16,https://www.getonbrd.com/jobs/analista-qa-auto...
96,full-stack-web-developer-remote-integrated-com...,Senior JavaScript Full-Stack Web Developer,A 4 or 5-year traditional Bachelor's degree in...,About ICS & the rolePowering Innovation for Hi...,"Utilize expert knowledge of JavaScript, includ...","Opportunity to work on complex, creative chall...",Experience with Client-side Database managemen...,"{'data': {'id': 4, 'type': 'seniority'}}",seniority,True,...,en,Programming,"[flexible_hours, informal_dresscode]",3800.0,4800.0,1715187249,NaN,NaN,91,https://www.getonbrd.com/jobs/full-stack-web-d...
97,back-end-developer-remote-integrated-co

Cuantas categorias hay en el dataset

In [14]:
unique_categories = df_jobs['category_name'].unique()

print("Categorias:", unique_categories)

print("Numero de categorias categorias:", len(unique_categories))


Categorias: ['SysAdmin / DevOps / QA' 'Programming' 'Machine Learning & AI'
 'Mobile Development' 'Data Science / Analytics'
 'Product, Innovation & Agile' 'Customer Support' 'Operations / Admin']
Numero de categorias categorias: 8


Trabajos total dataset

In [15]:
total_jobs = len(df_jobs)
print(total_jobs)

100


In [16]:
df_jobs.shape

(100, 22)

¿Cuál es la categoría con más y menos ofertas de trabajo?

In [17]:
categorias_frecuencia = df_jobs['category_name'].value_counts()
print(categorias_frecuencia)

category_name
Programming                    78
SysAdmin / DevOps / QA          9
Data Science / Analytics        4
Machine Learning & AI           2
Product, Innovation & Agile     2
Customer Support                2
Operations / Admin              2
Mobile Development              1
Name: count, dtype: int64


In [18]:
df_jobs.to_csv("try.csv", index=False)